In [1]:
!pip install -q matplotlib

In [1]:
# TODO: Change this as required
CHUNK_NUM = "02"
prefix = "shard_"
input_f = prefix + CHUNK_NUM
output_dir = 'outputs/'

output_f = output_dir  + "output_" + prefix + CHUNK_NUM + '.csv'

In [2]:
import pandas as pd
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import pickle
from typing import List

import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
df = pd.read_json(input_f, lines=True)

In [4]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters like &gt;
    text = re.sub(r'&\w+;', '', text)
    return text.lstrip('1234567890')

In [5]:
import time
import pickle

cnt = 0
total = 0

start_time = time.time()

partial_rows = []

for i in range(df.shape[0]):
  for j in range(len(df['posts'][i])):
    total += 1
    if 'perspectives' not in df['posts'][i][j] or 'com' not in df['posts'][i][j] or 'no' not in df['posts'][i][j] or 'now' not in df['posts'][i][j]:
      continue

    ps = df['posts'][i][j]['perspectives']
    c = clean_text(df['posts'][i][j]['com'])
    no = df['posts'][i][j]['no']
    ts = df['posts'][i][j]['now']

    if len(c) < 15:
      continue

    if max(ps.values()) >= 0.5:
      continue

    if 'http' in c:
      continue

    transformed_row = {k: v for k, v in ps.items()}
    transformed_row['com'] = c
    transformed_row['no'] = no
    transformed_row['ts'] = ts

    partial_rows.append(transformed_row)
    cnt += 1
    # print(ps)
    # print(c)
    # print(no)
    # print(ts)

transformed_df = pd.DataFrame(partial_rows)
transformed_df.to_csv(input_f + '_transformed.csv', index_label = 'id')

#pickle.dump(partial_rows, open(input_f + '_transformed.pkl', 'wb'))
end_time = time.time()

print(cnt)
print(total)

secs = end_time - start_time
per_entry = secs / total
print(f"For 100k entries it should take: {100000 * per_entry}")
print(f"Filtered out {100.0 * (total - cnt) / total} % of entries")


70673
241319
For 100k entries it should take: 5.5041470912428
Filtered out 70.71386836511009 % of entries


In [6]:
final_df = pd.read_csv(input_f + '_transformed.csv')
final_df

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts
0,0,0.075898,0.029977,0.197714,0.043075,0.080261,0.042533,0.222161,Is this politically relevant? I refuse to go s...,81179698,07/15/16(Fri)17:25:45
1,1,0.086786,0.045303,0.047925,0.051529,0.088246,0.068770,0.028049,"It was really good and funny, I hope the cast ...",81181225,07/15/16(Fri)17:31:56
2,2,0.093659,0.040851,0.224659,0.066957,0.079960,0.040760,0.110572,I think everyone keeps feeling like something ...,81144199,07/15/16(Fri)13:27:45
3,3,0.321328,0.342203,0.302661,0.274677,0.384605,0.076819,0.220402,The great 2016 Russian kebab removal?,81144334,07/15/16(Fri)13:29:06
4,4,0.207117,0.155102,0.371699,0.160154,0.248366,0.342150,0.136180,yeah WW3see you in Warsaw,81144461,07/15/16(Fri)13:30:20
...,...,...,...,...,...,...,...,...,...,...,...
70668,70668,0.297338,0.278793,0.189591,0.256574,0.250991,0.131370,0.212048,Among Wilkens’ injuries were “a broken clavicl...,81458796,07/17/16(Sun)12:34:03
70669,70669,0.032939,0.013367,0.225723,0.018680,0.022865,0.044236,0.017340,Well what can they do?,81458866,07/17/16(Sun)12:34:34
70670,70670,0.463585,0.305974,0.437721,0.365358,0.421443,0.136365,0.042755,"Not even kidding, he got off claiming that con...",81459171,07/17/16(Sun)12:36:47
70671,70671,0.116295,0.057756,0.233707,0.084386,0.122151,0.074929,0.086715,Unless you&#039;re in Alberta or Newfound land...,81459804,07/17/16(Sun)12:41:38


In [7]:
# Load the model and tokenizer once
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = '<pad>'
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

def get_dialogpt_response(comment):
    # Tokenize the input comment
    input_ids = tokenizer.encode(comment + tokenizer.eos_token, return_tensors="pt").to('cuda')

    # Generate a response
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

In [8]:
import pandas as pd

def process_and_save(final_df, func, b_chunk_size=200):
    """Applies a function to the DataFrame in chunks and saves periodically."""

    for i in range(0, len(final_df), b_chunk_size):
        chunk = final_df.iloc[i:i + b_chunk_size]
        chunk['response'] = chunk['com'].progress_apply(func)

        # Append to CSV file (mode='a' for append)
        chunk.to_csv(output_f, mode='a', header=i == 0, index=False)

process_and_save(final_df, get_dialogpt_response)

  0%|          | 0/200 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/scratch/1028977/ipykernel_2153626/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2153626/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

...

/scratch/1028977/ipykernel_2153626/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2153626/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

ValueError: Input length of input_ids is 1000, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
sanity = pd.read_csv(output_f)
sanity